In [1]:
import re
import os
import sys
import time
import codecs
import random
import shutil
import smtplib
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as BS
from pathlib import Path
from chardet import detect
from functools import reduce
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
%config Completer.use_jedi = False

In [3]:
def mkdirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

###  爬蟲之前

<span style="color:coral">先觀察</span>

### 開始囉

#### setting header

In [4]:
def _requests_session():
    session = requests.session()
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36"
    }
    session.headers.update(headers)
    return session

In [5]:
rs = _requests_session()

In [6]:
url = "https://www.dramexchange.com/"

In [7]:
res = rs.get(url)

In [8]:
res

<Response [200]>

In [9]:
soup = BS(res.text, "lxml")
soup

<!DOCTYPE html>
<html lang="english" xml:lang="english" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>DRAMeXchange - World leading DRAM and NAND Flash market research firm, with more than a decade of most authoritative database</title>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<meta content="width=device-width" name="viewport"/>
<meta content="DRAM, NAND Flash, memory card, SSD, DDR, DDR2, DDR3, Module, research, flash drive, memory stick, sdhc, SRAM, CPU, Spot, Contract price, Semiconductor, IC, Chip, EC, e-commerce, exchange, component, PC, Samsung, Micron, PSC, Intel, Sandisk" name="keywords"/>
<meta content="DRAM, NAND Flash, SSD, Module and Memory card, and provides market research on spot and contract prices, daily news, market views and reports, and monthly datasheets of semiconductor industry.集邦科技" name="description"/>
<script src="/bundles/jquery?v=JzhfglzUfmVF2qo-weTo-kvXJ9AJv

#### 找時間

In [10]:
soup.find("td", {"id": "NationalDramSpotPrice_show_day"})

<td class="title_right" id="NationalDramSpotPrice_show_day" style="height: 25px">
<span class="tab_time">Last Update: Apr.23 2021    18:10 (GMT+8) <a href="/Service/Spot_Price_Notice" target="blank"><span class="tab_notice">&lt;Price Notice&gt;</span></a></span>
</td>

In [11]:
soup.find(id="NationalDramSpotPrice_show_day")

<td class="title_right" id="NationalDramSpotPrice_show_day" style="height: 25px">
<span class="tab_time">Last Update: Apr.23 2021    18:10 (GMT+8) <a href="/Service/Spot_Price_Notice" target="blank"><span class="tab_notice">&lt;Price Notice&gt;</span></a></span>
</td>

In [12]:
soup.select("#NationalDramSpotPrice_show_day")

[<td class="title_right" id="NationalDramSpotPrice_show_day" style="height: 25px">
 <span class="tab_time">Last Update: Apr.23 2021    18:10 (GMT+8) <a href="/Service/Spot_Price_Notice" target="blank"><span class="tab_notice">&lt;Price Notice&gt;</span></a></span>
 </td>]

In [63]:
soup.find("td", {"id": "NationalDramSpotPrice_show_day"}).parent

<tr>
<td class="title_left" style="height: 25px">
<a href="/Price/Dram_Spot" rel="nofollow">
<font face="Arial, Helvetica, sans-serif">DRAM Spot Price</font>
<img align="absmiddle" border="0" src="/Common/Images/Home/more2.gif" style="padding-LEFT: 5px"/>
</a>
</td>
<td class="title_right" id="NationalDramSpotPrice_show_day" style="height: 25px">
<span class="tab_time">Last Update: Apr.23 2021    18:10 (GMT+8) <a href="/Service/Spot_Price_Notice" target="blank"><span class="tab_notice">&lt;Price Notice&gt;</span></a></span>
</td>
</tr>

https://www.w3schools.com/jquery/jquery_selectors.asp

#### 取時間

https://docs.python.org/3/library/time.html#time.strftime

In [13]:
soup.find(id="NationalDramSpotPrice_show_day").text

'\nLast Update: Apr.23 2021    18:10 (GMT+8) <Price Notice>\n'

In [14]:
soup.find(id="NationalDramSpotPrice_show_day").contents[1]

<span class="tab_time">Last Update: Apr.23 2021    18:10 (GMT+8) <a href="/Service/Spot_Price_Notice" target="blank"><span class="tab_notice">&lt;Price Notice&gt;</span></a></span>

In [15]:
soup.find(id="NationalDramSpotPrice_show_day").contents[1].contents[0]

'Last Update: Apr.23 2021    18:10 (GMT+8) '

In [16]:
soup.find(id="NationalDramSpotPrice_show_day").contents[1].text

'Last Update: Apr.23 2021    18:10 (GMT+8) <Price Notice>'

In [17]:
text_time = soup.find(id="NationalDramSpotPrice_show_day").text

In [18]:
str_time = re.findall("\w+[\. ]?\d+ \d{4}", text_time)[0]
str_time

'Apr.23 2021'

In [19]:
str_time.replace(".", "")

'Apr23 2021'

In [20]:
time_dram = time.strptime(str_time.replace(".", ""), "%b%d %Y")
time_dram

time.struct_time(tm_year=2021, tm_mon=4, tm_mday=23, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=4, tm_yday=113, tm_isdst=-1)

https://stackoverflow.com/questions/12255932/what-does-the-p-in-strptime-stand-for

In [21]:
date_dram = time.strftime("%Y-%m-%d", time_dram)
date_dram

'2021-04-23'

#### 找table

In [22]:
soup.find_all("table")

[<table align="center" border="0" cellpadding="3" cellspacing="0" width="1000">
 <tbody>
 <tr>
 <td rowspan="3" valign="top" width="231">
 <div align="right" style="padding-top: 20px;">
 <h1 style="list-style: none; margin: 0 0 10px 0;"><a href="/">
 <img alt="DRAMeXchange" border="0" height="66" src="/Common/Images/Header/logo.gif" title="DRAM, NAND Flash price, DRAMeXchange" width="226"/></a></h1>
 <a href="/Member/Login" rel="nofollow">
 <img alt="login" border="0" hspace="4" src="/Common/Images/Header/login.gif"/></a>
 <a href="/Member/RegisterPre" rel="nofollow"><img alt="purchase" border="0" src="/Common/Images/Header/register.gif"/></a><a href="/Intelligence/Membership"><img alt="purchase" border="0" src="/Common/Images/Header/mi_membertop.gif"/></a>
 </div>
 </td>
 <td colspan="3" valign="top">
 <div align="right">
 <a class="origin" href="//www.trendforce.com/" rel="nofollow" target="_blank">TrendForce:</a>
 <a class="origin" href="//www.dramexchange.com/" rel="nofollow">Semic

In [23]:
soup.find("tbody", {"id": "tb_NationalDramSpotPrice"}).parent

<table border="0" cellpadding="0" cellspacing="0" width="100%">
<tbody id="tb_NationalDramSpotPrice">
<tr>
<td class="tab_title" width="37%">Item</td>
<td class="tab_title" width="8%">Daily High</td>
<td class="tab_title" width="8%">Daily Low</td>
<td class="tab_title" width="9%">Session High</td>
<td class="tab_title" width="9%">Session Low</td>
<td class="tab_title" width="9%">Session Average</td>
<td class="tab_title" width="12%">Session Change</td>
<td class="tab_title" width="8%">History</td>
</tr>
<tr>
<td class="tab_tr_gray2">
<a href="/Price/Dram_Spot" original-title="Samsung, SK Hynix">
<img border="0" src="/Common/Images/chip.gif" style="vertical-align: middle;"/>
                                                    DDR4 8G (1G*8) 2400 Mbps
                                                </a>
</td>
<td class="tab_tr_gray">4.60</td>
<td class="tab_tr_gray">4.25</td>
<td class="tab_tr_gray">4.60</td>
<td class="tab_tr_gray">4.25</td>
<td class="tab_tr_gray">4.405</td>
<td class=

In [24]:
soup.find_all("tbody", {"id": "tb_NationalDramSpotPrice"})[0].parent

<table border="0" cellpadding="0" cellspacing="0" width="100%">
<tbody id="tb_NationalDramSpotPrice">
<tr>
<td class="tab_title" width="37%">Item</td>
<td class="tab_title" width="8%">Daily High</td>
<td class="tab_title" width="8%">Daily Low</td>
<td class="tab_title" width="9%">Session High</td>
<td class="tab_title" width="9%">Session Low</td>
<td class="tab_title" width="9%">Session Average</td>
<td class="tab_title" width="12%">Session Change</td>
<td class="tab_title" width="8%">History</td>
</tr>
<tr>
<td class="tab_tr_gray2">
<a href="/Price/Dram_Spot" original-title="Samsung, SK Hynix">
<img border="0" src="/Common/Images/chip.gif" style="vertical-align: middle;"/>
                                                    DDR4 8G (1G*8) 2400 Mbps
                                                </a>
</td>
<td class="tab_tr_gray">4.60</td>
<td class="tab_tr_gray">4.25</td>
<td class="tab_tr_gray">4.60</td>
<td class="tab_tr_gray">4.25</td>
<td class="tab_tr_gray">4.405</td>
<td class=

In [25]:
table_dram = soup.find("tbody", {"id": "tb_NationalDramSpotPrice"}).parent

In [26]:
pd.read_html(table_dram.prettify(), header=None)

[                                 0           1          2             3  \
 0                             Item  Daily High  Daily Low  Session High   
 1         DDR4 8G (1G*8) 2400 Mbps        4.60       4.25          4.60   
 2       DDR4 4G (512M*8) 2400 Mbps        4.05       2.55          4.05   
 3   DDR3  4Gb 512Mx8 1600/1866Mbps        3.95       2.70          3.95   
 4             DDR3  4Gb 512Mx8 eTT        2.25       1.90          2.25   
 5  DDR3  4Gb 256Mx16 1600/1866Mbps        3.50       3.10          3.50   
 6   DDR3  2Gb 256Mx8 1600/1866Mbps        4.05       2.55          4.05   
 
              4                5               6        7  
 0  Session Low  Session Average  Session Change  History  
 1         4.25            4.405          0.11 %      NaN  
 2         2.55            3.240          0.00 %      NaN  
 3         2.70            3.401          0.00 %      NaN  
 4         1.90            2.065         -0.48 %      NaN  
 5         3.10            3.3

In [27]:
pd.read_html(table_dram.prettify(), header=None)[0]

,0,1,2,3,4,5,6,7
0,Item,Daily High,Daily Low,Session High,Session Low,Session Average,Session Change,History
1,DDR4 8G (1G*8) 2400 Mbps,4.60,4.25,4.60,4.25,4.405,0.11 %,NaN
2,DDR4 4G (512M*8) 2400 Mbps,4.05,2.55,4.05,2.55,3.240,0.00 %,NaN
3,DDR3 4Gb 512Mx8 1600/1866Mbps,3.95,2.70,3.95,2.70,3.401,0.00 %,NaN
4,DDR3 4Gb 512Mx8 eTT,2.25,1.90,2.25,1.90,2.065,-0.48 %,NaN
5,DDR3 4Gb 256Mx16 1600/1866Mbps,3.50,3.10,3.50,3.10,3.324,-0.09 %,NaN
6,DDR3 2Gb 256Mx8 1600/1866Mbps,4.05,2.55,4.05,2.55,2.995,0.00 %,NaN


In [28]:
df_dram = pd.read_html(table_dram.prettify(), header=0)[0]
df_dram

,Item,Daily High,Daily Low,Session High,Session Low,Session Average,Session Change,History
0,DDR4 8G (1G*8) 2400 Mbps,4.60,4.25,4.60,4.25,4.405,0.11 %,NaN
1,DDR4 4G (512M*8) 2400 Mbps,4.05,2.55,4.05,2.55,3.240,0.00 %,NaN
2,DDR3 4Gb 512Mx8 1600/1866Mbps,3.95,2.70,3.95,2.70,3.401,0.00 %,NaN
3,DDR3 4Gb 512Mx8 eTT,2.25,1.90,2.25,1.90,2.065,-0.48 %,NaN
4,DDR3 4Gb 256Mx16 1600/1866Mbps,3.50,3.10,3.50,3.10,3.324,-0.09 %,NaN
5,DDR3 2Gb 256Mx8 1600/1866Mbps,4.05,2.55,4.05,2.55,2.995,0.00 %,NaN


In [29]:
df_dram_daily = df_dram[["Item", "Session Average"]]
df_dram_daily

,Item,Session Average
0,DDR4 8G (1G*8) 2400 Mbps,4.405
1,DDR4 4G (512M*8) 2400 Mbps,3.240
2,DDR3 4Gb 512Mx8 1600/1866Mbps,3.401
3,DDR3 4Gb 512Mx8 eTT,2.065
4,DDR3 4Gb 256Mx16 1600/1866Mbps,3.324
5,DDR3 2Gb 256Mx8 1600/1866Mbps,2.995


#### Pandas

In [30]:
cols = ["Item", date_dram]
cols

['Item', '2021-04-23']

In [31]:
df_dram_daily.columns = cols
df_dram_daily

,Item,2021-04-23
0,DDR4 8G (1G*8) 2400 Mbps,4.405
1,DDR4 4G (512M*8) 2400 Mbps,3.240
2,DDR3 4Gb 512Mx8 1600/1866Mbps,3.401
3,DDR3 4Gb 512Mx8 eTT,2.065
4,DDR3 4Gb 256Mx16 1600/1866Mbps,3.324
5,DDR3 2Gb 256Mx8 1600/1866Mbps,2.995


In [32]:
file_name = "dram-{}.csv".format(date_dram)
file_name

'dram-2021-04-23.csv'

In [33]:
folder_path = "./dram-daily"
folder_path

'./dram-daily'

In [34]:
mkdirs(folder_path)

In [35]:
file_path = "{}/{}".format(folder_path, file_name)
file_path

'./dram-daily/dram-2021-04-23.csv'

In [36]:
df_dram_daily.to_csv(file_path, index=False, encoding="utf-8-sig")

In [37]:
folder_merge = "./dram-merge"
folder_merge

'./dram-merge'

In [38]:
mkdirs(folder_merge)

In [39]:
df_dram_daily.to_csv(merge_path_0, index=False, encoding="utf-8-sig")

NameError: name 'merge_path_0' is not defined

#### 合併新的table

https://pandas.pydata.org/docs/user_guide/merging.html

In [41]:
os.listdir(folder_merge)[-1]

'dram-merge-2021-04-06.csv'

In [65]:
date_dram

'2021-04-23'

In [ ]:
os.listdir(folder_merge)[-1]

In [80]:
list((Path(folder_merge).iterdir()))[0].name

'dram-merge-2021-04-23.csv'

In [79]:
list((Path(folder_merge).iterdir()))[0].stem

'dram-merge-2021-04-23'

In [87]:
list((Path(folder_merge).iterdir()))[0].suffix

'.csv'

In [90]:
date_dram not in list((Path(folder_merge).iterdir()))[0].stem

False

In [42]:
merge_path_0 = "{}/{}".format(folder_merge, os.listdir(folder_merge)[-1])
merge_path_0

'./dram-merge/dram-merge-2021-04-06.csv'

In [43]:
df_0 = pd.read_csv(merge_path_0, encoding="utf-8")
df_0

,Item,2021-04-06
0,DDR4 8G (1G*8) 2400 Mbps,4.175
1,DDR4 4G (512M*8) 2400 Mbps,3.250
2,DDR3 4Gb 512Mx8 1600/1866Mbps,3.280
3,DDR3 4Gb 512Mx8 eTT,2.080
4,DDR3 4Gb 256Mx16 1600/1866Mbps,3.417
5,DDR3 2Gb 256Mx8 1600/1866Mbps,2.926


In [44]:
df_dram_daily

,Item,2021-04-23
0,DDR4 8G (1G*8) 2400 Mbps,4.405
1,DDR4 4G (512M*8) 2400 Mbps,3.240
2,DDR3 4Gb 512Mx8 1600/1866Mbps,3.401
3,DDR3 4Gb 512Mx8 eTT,2.065
4,DDR3 4Gb 256Mx16 1600/1866Mbps,3.324
5,DDR3 2Gb 256Mx8 1600/1866Mbps,2.995


In [45]:
df_0.append(df_dram_daily)

,Item,2021-04-06,2021-04-23
0,DDR4 8G (1G*8) 2400 Mbps,4.175,NaN
1,DDR4 4G (512M*8) 2400 Mbps,3.250,NaN
2,DDR3 4Gb 512Mx8 1600/1866Mbps,3.280,NaN
3,DDR3 4Gb 512Mx8 eTT,2.080,NaN
4,DDR3 4Gb 256Mx16 1600/1866Mbps,3.417,NaN
5,DDR3 2Gb 256Mx8 1600/1866Mbps,2.926,NaN
0,DDR4 8G (1G*8) 2400 Mbps,NaN,4.405
1,DDR4 4G (512M*8) 2400 Mbps,NaN,3.240
2,DDR3 4Gb 512Mx8 1600/1866Mbps,NaN,3.401
3,DDR3 4Gb 512Mx8 eTT,NaN,2.065


In [46]:
pd.concat([df_0, df_dram_daily], axis=1)

,Item,2021-04-06,Item,2021-04-23
0,DDR4 8G (1G*8) 2400 Mbps,4.175,DDR4 8G (1G*8) 2400 Mbps,4.405
1,DDR4 4G (512M*8) 2400 Mbps,3.250,DDR4 4G (512M*8) 2400 Mbps,3.240
2,DDR3 4Gb 512Mx8 1600/1866Mbps,3.280,DDR3 4Gb 512Mx8 1600/1866Mbps,3.401
3,DDR3 4Gb 512Mx8 eTT,2.080,DDR3 4Gb 512Mx8 eTT,2.065
4,DDR3 4Gb 256Mx16 1600/1866Mbps,3.417,DDR3 4Gb 256Mx16 1600/1866Mbps,3.324
5,DDR3 2Gb 256Mx8 1600/1866Mbps,2.926,DDR3 2Gb 256Mx8 1600/1866Mbps,2.995


In [47]:
pd.merge(df_0, df_dram_daily)

,Item,2021-04-06,2021-04-23
0,DDR4 8G (1G*8) 2400 Mbps,4.175,4.405
1,DDR4 4G (512M*8) 2400 Mbps,3.250,3.240
2,DDR3 4Gb 512Mx8 1600/1866Mbps,3.280,3.401
3,DDR3 4Gb 512Mx8 eTT,2.080,2.065
4,DDR3 4Gb 256Mx16 1600/1866Mbps,3.417,3.324
5,DDR3 2Gb 256Mx8 1600/1866Mbps,2.926,2.995


In [48]:
df_merge = pd.merge(df_0, df_dram_daily)

In [49]:
merge_file = "dram-merge-{}.csv".format(date_dram)
merge_file

'dram-merge-2021-04-23.csv'

In [50]:
merge_path = "{}/{}".format(folder_merge, merge_file)
merge_path

'./dram-merge/dram-merge-2021-04-23.csv'

In [51]:
df_merge.to_csv(merge_path, index=False, encoding="utf-8-sig")

舊的不想留

In [52]:
a = os.listdir(folder_merge)
a

['dram-merge-2021-04-06.csv', 'dram-merge-2021-04-23.csv']

In [56]:
str(list((Path(folder_merge).iterdir()))[0])

'dram-merge\\dram-merge-2021-04-06.csv'

In [57]:
list((Path(folder_merge).iterdir()))[0].name

'dram-merge-2021-04-06.csv'

In [59]:
list(Path(folder_merge).glob("*"))

[WindowsPath('dram-merge/dram-merge-2021-04-06.csv'),
 WindowsPath('dram-merge/dram-merge-2021-04-23.csv')]

In [60]:
b = Path(folder_merge).iterdir()
b

<generator object Path.iterdir at 0x0000019F680F8DD0>

In [61]:
c = [x for x in b if x.name != merge_file]
c

[WindowsPath('dram-merge/dram-merge-2021-04-06.csv')]

In [62]:
for i in c:
    Path(i).unlink(missing_ok=True)

統整起來

In [91]:
url = "https://www.dramexchange.com/"
rs = _requests_session()
res = rs.get(url)
soup = BS(res.text, "lxml")
soup.find("td", {"id": "NationalDramSpotPrice_show_day"})
text_time = soup.find(id="NationalDramSpotPrice_show_day").text
str_time = re.findall("\w+[\. ]?\d+ \d{4}", text_time)[0]
time_dram = time.strptime(str_time.replace(".", ""), "%b%d %Y")
date_dram = time.strftime("%Y-%m-%d", time_dram)
table_dram = soup.find("tbody", {"id": "tb_NationalDramSpotPrice"}).parent
df_dram = pd.read_html(table_dram.prettify(), header=0)[0]
df_dram_daily = df_dram[["Item", "Session Average"]]
cols = ["Item", date_dram]
df_dram_daily.columns = cols
file_name = "dram-{}.csv".format(date_dram)
folder_path = "./dram-daily"
mkdirs(folder_path)
file_path = "{}/{}".format(folder_path, file_name)
df_dram_daily.to_csv(file_path, index=False, encoding="utf-8-sig")
folder_merge = "./dram-merge"
mkdirs(folder_merge)
if date_dram in list((Path(folder_merge).iterdir()))[0].stem:
    return
merge_path_0 = "{}/{}".format(folder_merge, os.listdir(folder_merge)[-1])
df_0 = pd.read_csv(merge_path_0, encoding="utf-8")
df_merge = pd.merge(df_0, df_dram_daily)
merge_file = "dram-merge-{}.csv".format(date_dram)
merge_path = "{}/{}".format(folder_merge, merge_file)
df_merge.to_csv(merge_path, index=False, encoding="utf-8-sig")

SyntaxError: 'return' outside function (<ipython-input-91-7f04e8f1a689>, line 23)

In [96]:
def extract_table():
    url = "https://www.dramexchange.com/"
    rs = _requests_session()
    res = rs.get(url)
    soup = BS(res.text, "lxml")
    soup.find("td", {"id": "NationalDramSpotPrice_show_day"})
    text_time = soup.find(id="NationalDramSpotPrice_show_day").text
    str_time = re.findall("\w+[\. ]?\d+ \d{4}", text_time)[0]
    time_dram = time.strptime(str_time.replace(".", ""), "%b%d %Y")
    date_dram = time.strftime("%Y-%m-%d", time_dram)
    table_dram = soup.find("tbody", {"id": "tb_NationalDramSpotPrice"}).parent
    df_dram = pd.read_html(table_dram.prettify(), header=0)[0]
    df_dram_daily = df_dram[["Item", "Session Average"]]
    cols = ["Item", date_dram]
    df_dram_daily.columns = cols
    file_name = "dram-{}.csv".format(date_dram)
    folder_path = "./dram-daily"
    mkdirs(folder_path)
    file_path = "{}/{}".format(folder_path, file_name)
    df_dram_daily.to_csv(file_path, index=False, encoding="utf-8-sig")
    folder_merge = "./dram-merge"
    mkdirs(folder_merge)
    if date_dram in list((Path(folder_merge).iterdir()))[0].stem:
        print("merged table existed!")
        return
    merge_path_0 = "{}/{}".format(folder_merge, os.listdir(folder_merge)[-1])
    df_0 = pd.read_csv(merge_path_0, encoding="utf-8")
    df_merge = pd.merge(df_0, df_dram_daily)
    merge_file = "dram-merge-{}.csv".format(date_dram)
    merge_path = "{}/{}".format(folder_merge, merge_file)
    df_merge.to_csv(merge_path, index=False, encoding="utf-8-sig")

In [97]:
extract_table()

merged table existed!
